In [254]:
import numpy as np
import pandas as pd
import requests # 웹 상의 데이터를 가져올 때
from bs4 import BeautifulSoup # 가져온 데이터에서 필요한 정보를 골라낼(파싱할) 때

# 확인

In [2]:
# 셀레니움 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [256]:
# 크롬창 열기
driver = webdriver.Chrome('./chromedriver.exe')

In [284]:
# 설정한 url로 데이터 get함수로 요청
# 검색지역 지정
keyword = "속초"

# url정의
url = f'https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page=1'
driver.get(url)

In [258]:
# 첫 번째 게시물 클릭
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img').click()

In [285]:
# 무분별한 크롤링을 막기위한 방지코드가 존재
# 아래 사이트에서 User-Agent 정보를 복사하여 header를 추가하여 request한다.
# www.useragentstring.com/
# headers = {'User-Agent': '본인의 User-Agent'} # 각자 본인의 User-Agent 정보 입력
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'} # 각자 본인의 User-Agent 정보 입력 

# requsts로 url에 정보요청 (.get 함수를 자주 사용하면 서비스 제공자가 블락할 수 있어 1번 사용한 후, 주석처리를 하고 변수로 설정하여 사용한다.)
resp = requests.get(url, headers = headers)

# 정보를 html 변환 (보기 쉽게)
soup = BeautifulSoup(resp.text, 'lxml')

In [260]:
# 원하는 항목의 데이터만 추출 
address = soup.select('img')[0]['alt']

In [263]:
soup.select('img')[9]['alt']

'중앙닭강정 사진 - 강원도 속초시 중앙동 471-4'

In [264]:
title = soup.select("h2.title")[0].text
search_point = soup.select("strong.point.search_point")[0].text
etc = soup.select("p.etc")[0].text
view_count = soup.select("span.view_count")[0].text
review_count = soup.select("span.review_count")[0].text

In [266]:
title, search_point, etc, view_count, review_count, address

('모녀가리비\n',
 '4.5',
 '강원 속초시 - 해산물 요리',
 '4,174',
 '10',
 '모녀가리비 사진 - 강원도 속초시 대포동 964-13')

In [ ]:
# 첫 번째 사진
/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img

In [ ]:
# 두 번째 사진
/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[2]/figure/a/div/img

# 실습

In [342]:
# 빈 딕셔너리 생성
dic_ = {
    'title': [],
    'search_point': [],
    'etc': [],
    'view_count': [],
    'review_count': [],
    'address' : []
}

# 검색지역 지정
keyword = "속초"

# 1~50 페이지 설정
# 숨김페이지 50페이지
for page in range(50):
    # url정의
    url = f'https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page={page}'

    # 무분별한 크롤링을 막기위한 방지코드가 존재
    # 아래 사이트에서 User-Agent 정보를 복사하여 header를 추가하여 request한다.
    # www.useragentstring.com/
    # headers = {'User-Agent': '본인의 User-Agent'} # 각자 본인의 User-Agent 정보 입력
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'} # 각자 본인의 User-Agent 정보 입력 
    
    # requsts로 url에 정보요청 (.get 함수를 자주 사용하면 서비스 제공자가 블락할 수 있어 1번 사용한 후, 주석처리를 하고 변수로 설정하여 사용한다.)
    resp = requests.get(url, headers = headers)

    # 정보를 html 변환 (보기 쉽게)
    soup = BeautifulSoup(resp.text, 'lxml')

    # 20개의 게시물 크롤링
    for i in range(20):
        title = soup.select("h2.title")[i].text
        search_point = soup.select("strong.point.search_point")[i].text
        etc = soup.select("p.etc")[i].text
        view_count = soup.select("span.view_count")[i].text
        review_count = soup.select("span.review_count")[i].text
        address = soup.select('img')[i]['alt']
        
        # address에서 'title'과 "사진 -" 문자열 제거(문제)
        # address에서 강원도 -> 원도로 되는 문제 발생
        # .lstrip(title + " 사진")으로 "-"는 남겨둔다.
        address = address.lstrip(title + " 사진")
        address = address.replace("- ", "")

        dic_['title'].append(title)
        dic_['search_point'].append(search_point)
        dic_['etc'].append(etc)
        dic_['view_count'].append(view_count)
        dic_['review_count'].append(review_count)
        dic_['address'].append(address)

In [276]:
# 'address' 필요없는 문자열 제거
title, address

('카페시오\n', '강원도 속초시 중앙동 471-4')

In [289]:
address.lstrip("-")

'강원도 속초시 중앙동 471-4'

In [281]:
# address에서 강원도 -> 원도로 되는 문제 발생
dic_["address"]

['강원도 속초시 대포동 964-13',
 '강원도 속초시 중앙동 472-5',
 '강원도 속초시 청호동 842',
 '강원도 속초시 장사동 525-3',
 '강원도 속초시 교동 780-55',
 '강원도 속초시 조양동 1287-12',
 '강원도 속초시 중앙동 468-34',
 '강원도 속초시 중앙동 479-29',
 '강원도 속초시 영랑동 133',
 '원도 속초시 중앙동 471-4',
 '강원도 속초시 영랑동 131-19',
 '강원도 속초시 청호동 918',
 '강원도 속초시 중앙동 475-60',
 '강원도 속초시 노학동 1015-2',
 '강원도 속초시 청호동 870',
 '강원도 속초시 청호동 1329',
 '강원도 속초시 조양동 1555-1',
 '강원도 속초시 영랑동 148-30',
 '강원도 속초시 대포동 780',
 '강원도 속초시 중앙동 478-49',
 '강원도 속초시 대포동 964-13',
 '강원도 속초시 중앙동 472-5',
 '강원도 속초시 청호동 842',
 '강원도 속초시 장사동 525-3',
 '강원도 속초시 교동 780-55',
 '강원도 속초시 조양동 1287-12',
 '강원도 속초시 중앙동 468-34',
 '강원도 속초시 중앙동 479-29',
 '강원도 속초시 영랑동 133',
 '원도 속초시 중앙동 471-4',
 '강원도 속초시 영랑동 131-19',
 '강원도 속초시 청호동 918',
 '강원도 속초시 중앙동 475-60',
 '강원도 속초시 노학동 1015-2',
 '강원도 속초시 청호동 870',
 '강원도 속초시 청호동 1329',
 '강원도 속초시 조양동 1555-1',
 '강원도 속초시 영랑동 148-30',
 '강원도 속초시 대포동 780',
 '강원도 속초시 중앙동 478-49',
 '강원도 속초시 영랑동 129',
 '강원도 속초시 영랑동 204-1',
 '강원도 속초시 노학동 972-120',
 '강원도 속초시 노학동 1011-39',
 '강원도 속초시 교동 933-1',
 '강원도 속초시 조

In [320]:
b = soup.select('img')[8]['alt']

In [321]:
b

'사돈집 사진 - 강원도 속초시 영랑동 133'

In [312]:
a = soup.select('img')[9]['alt']

In [313]:
a

'중앙닭강정 사진 - 강원도 속초시 중앙동 471-4'

In [343]:
c = b.lstrip("사돈집" + " 사진 - ")
c

'강원도 속초시 영랑동 133'

In [338]:
c.replace("- ","")

'강원도 속초시 영랑동 133'

In [348]:
d = a.lstrip("중앙닭강정" + " 사진 - ")
d

'원도 속초시 중앙동 471-4'

In [340]:
d.replace("- ", "")

'강원도 속초시 중앙동 471-4'

In [349]:
# dic_['title']의 "\n" 과 "" 처리 필요
dic_['title']

['모녀가리비\n',
 '속초붉은대게수산\n',
 '단천식당\n',
 '왕박골식당\n',
 '속초751샌드위치\n',
 '남경막국수\n',
 '송도횟집\n                                    (본점)\n',
 '마카오박\n',
 '사돈집\n',
 '중앙닭강정\n                                    (본점)\n',
 '속초생대구\n',
 '북청전통아바이순대\n',
 '총각네대게\n',
 '초원순두부\n',
 '아바이옛날식당\n',
 '아루나\n',
 '청초수물회\n                                    (속초본점)\n',
 '봉포머구리집\n',
 '팔팔회센타\n                                    (본점)\n',
 '엄지네포장마차\n                                    (속초점)\n',
 '모녀가리비\n',
 '속초붉은대게수산\n',
 '단천식당\n',
 '왕박골식당\n',
 '속초751샌드위치\n',
 '남경막국수\n',
 '송도횟집\n                                    (본점)\n',
 '마카오박\n',
 '사돈집\n',
 '중앙닭강정\n                                    (본점)\n',
 '속초생대구\n',
 '북청전통아바이순대\n',
 '총각네대게\n',
 '초원순두부\n',
 '아바이옛날식당\n',
 '아루나\n',
 '청초수물회\n                                    (속초본점)\n',
 '봉포머구리집\n',
 '팔팔회센타\n                                    (본점)\n',
 '엄지네포장마차\n                                    (속초점)\n',
 '황가네찜\n',
 '그리운보리밥\n',
 '섭죽마을\n',
 '김영애할머니순두부\n                            

In [350]:
df = pd.DataFrame(dic_)
df

,title,search_point,etc,view_count,review_count,address
0,모녀가리비\n,4.5,강원 속초시 - 해산물 요리,"4,174",10,강원도 속초시 대포동 964-13
1,속초붉은대게수산\n,4.4,강원 속초시 - 해산물 요리,"34,739",17,강원도 속초시 중앙동 472-5
2,단천식당\n,4.3,강원 속초시 - 탕 / 찌개 / 전골,"30,457",38,강원도 속초시 청호동 842
3,왕박골식당\n,4.3,강원 속초시 - 국수 / 면 요리,"24,858",14,강원도 속초시 장사동 525-3
4,속초751샌드위치\n,4.3,강원 속초시 - 브런치 / 버거 / 샌드위치,"16,463",14,강원도 속초시 교동 780-55
5,남경막국수\n,4.3,강원 속초시 - 국수 / 면 요리,"26,067",14,강원도 속초시 조양동 1287-12
6,송도횟집\n (본점)\n,4.3,강원 속초시 - 해산물 요리,"30,265",13,강원도 속초시 중앙동 468-34
7,마카오박\n,4.3,강원 속초시 - 카페 / 디저트,973,9,강원도 속초시 중앙동 479-29
8,사돈집\n,4.3,강원 속초시 - 탕 / 찌개 / 전골,"8,042",9,강원도 속초시 영랑동 133
9,중앙닭강정\n (본점)\n,4.2,강원 속초시 - 닭 / 오리 요리,"88,477",90,강원도 속초시 중앙동 471-4


In [272]:
df["address"][9]

'원도 속초시 중앙동 471-4'

In [273]:
pd.set_option('display.max_rows', None) ## 모든 행을 출력한다.
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

In [351]:
# 줄 바꿈기호(\n) 삭제
df['title'] = df["title"].replace("\n", "", regex=True)

In [352]:
# 확인
df["title"]

0                                                  모녀가리비
1                                               속초붉은대게수산
2                                                   단천식당
3                                                  왕박골식당
4                                              속초751샌드위치
5                                                  남경막국수
6           송도횟집                                    (본점)
7                                                   마카오박
8                                                    사돈집
9          중앙닭강정                                    (본점)
10                                                 속초생대구
11                                             북청전통아바이순대
12                                                 총각네대게
13                                                 초원순두부
14                                               아바이옛날식당
15                                                   아루나
16       청초수물회                                    (속초본점)
17                             

In [353]:
# 빈칸 제거
df['title'] = df["title"].replace(" ", "", regex=True)

In [354]:
df

,title,search_point,etc,view_count,review_count,address
0,모녀가리비,4.5,강원 속초시 - 해산물 요리,"4,174",10,강원도 속초시 대포동 964-13
1,속초붉은대게수산,4.4,강원 속초시 - 해산물 요리,"34,739",17,강원도 속초시 중앙동 472-5
2,단천식당,4.3,강원 속초시 - 탕 / 찌개 / 전골,"30,457",38,강원도 속초시 청호동 842
3,왕박골식당,4.3,강원 속초시 - 국수 / 면 요리,"24,858",14,강원도 속초시 장사동 525-3
4,속초751샌드위치,4.3,강원 속초시 - 브런치 / 버거 / 샌드위치,"16,463",14,강원도 속초시 교동 780-55
5,남경막국수,4.3,강원 속초시 - 국수 / 면 요리,"26,067",14,강원도 속초시 조양동 1287-12
6,송도횟집(본점),4.3,강원 속초시 - 해산물 요리,"30,265",13,강원도 속초시 중앙동 468-34
7,마카오박,4.3,강원 속초시 - 카페 / 디저트,973,9,강원도 속초시 중앙동 479-29
8,사돈집,4.3,강원 속초시 - 탕 / 찌개 / 전골,"8,042",9,강원도 속초시 영랑동 133
9,중앙닭강정(본점),4.2,강원 속초시 - 닭 / 오리 요리,"88,477",90,강원도 속초시 중앙동 471-4


In [355]:
# title에 "속초"가 들어간 경우가 있어 제거
df = df[df["etc"].str.contains("강원 속초시")]

In [248]:
# index를 재설정하면서 이전 index는 삭제
df.reset_index(drop=True, inplace=True)

In [356]:
# 중복 row 제거
df.drop_duplicates(ignore_index = True, inplace=True)

<ipython-input-356-53c86a37f1ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(ignore_index = True, inplace=True)


In [357]:
df

,title,search_point,etc,view_count,review_count,address
0,모녀가리비,4.5,강원 속초시 - 해산물 요리,"4,174",10,강원도 속초시 대포동 964-13
1,속초붉은대게수산,4.4,강원 속초시 - 해산물 요리,"34,739",17,강원도 속초시 중앙동 472-5
2,단천식당,4.3,강원 속초시 - 탕 / 찌개 / 전골,"30,457",38,강원도 속초시 청호동 842
3,왕박골식당,4.3,강원 속초시 - 국수 / 면 요리,"24,858",14,강원도 속초시 장사동 525-3
4,속초751샌드위치,4.3,강원 속초시 - 브런치 / 버거 / 샌드위치,"16,463",14,강원도 속초시 교동 780-55
5,남경막국수,4.3,강원 속초시 - 국수 / 면 요리,"26,067",14,강원도 속초시 조양동 1287-12
6,송도횟집(본점),4.3,강원 속초시 - 해산물 요리,"30,265",13,강원도 속초시 중앙동 468-34
7,마카오박,4.3,강원 속초시 - 카페 / 디저트,973,9,강원도 속초시 중앙동 479-29
8,사돈집,4.3,강원 속초시 - 탕 / 찌개 / 전골,"8,042",9,강원도 속초시 영랑동 133
9,중앙닭강정(본점),4.2,강원 속초시 - 닭 / 오리 요리,"88,477",90,강원도 속초시 중앙동 471-4


In [358]:
# serch_point = "" -> 0 으로 변환
df["search_point"].replace("", 0.0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [359]:
df

,title,search_point,etc,view_count,review_count,address
0,모녀가리비,4.5,강원 속초시 - 해산물 요리,"4,174",10,강원도 속초시 대포동 964-13
1,속초붉은대게수산,4.4,강원 속초시 - 해산물 요리,"34,739",17,강원도 속초시 중앙동 472-5
2,단천식당,4.3,강원 속초시 - 탕 / 찌개 / 전골,"30,457",38,강원도 속초시 청호동 842
3,왕박골식당,4.3,강원 속초시 - 국수 / 면 요리,"24,858",14,강원도 속초시 장사동 525-3
4,속초751샌드위치,4.3,강원 속초시 - 브런치 / 버거 / 샌드위치,"16,463",14,강원도 속초시 교동 780-55
5,남경막국수,4.3,강원 속초시 - 국수 / 면 요리,"26,067",14,강원도 속초시 조양동 1287-12
6,송도횟집(본점),4.3,강원 속초시 - 해산물 요리,"30,265",13,강원도 속초시 중앙동 468-34
7,마카오박,4.3,강원 속초시 - 카페 / 디저트,973,9,강원도 속초시 중앙동 479-29
8,사돈집,4.3,강원 속초시 - 탕 / 찌개 / 전골,"8,042",9,강원도 속초시 영랑동 133
9,중앙닭강정(본점),4.2,강원 속초시 - 닭 / 오리 요리,"88,477",90,강원도 속초시 중앙동 471-4


In [360]:
# 저장
df.to_csv("속초_맛집.csv", encoding='cp949', index=False)